# Tutorial

In [1]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from konlpy.tag import Okt

import re
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

import argparse
import wandb
wandb.init(project="DACON_235938", name="tutorial")

parser = argparse.ArgumentParser(description="tutorial")
parser.add_argument('--vectorizer', default="tfidf", type=str) # count or hashing or tfidf
parser.add_argument('--feature_name', default="tokenized_stem", type=str) # reviews or tokenized_stem or main_pos
parser.add_argument('--max_features', default=None, type=int)
parser.add_argument('--validation_size', default=0.2, type=float)
parser.add_argument('--seed', default=1011, type=int)
args = parser.parse_args('')

wandb.config.update(args)

vectorizer = args.vectorizer
feature_name = args.feature_name
max_features = args.max_features
validation_size = args.validation_size
seed = args.seed

def set_seeds(seed=seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)

set_seeds()

train = pd.read_csv("data/train_okt.csv")
test = pd.read_csv("data/test_okt.csv")

train.head()

wandb: Currently logged in as: gnoeyheat (use `wandb login --relogin` to force relogin)


,id,reviews,target,tokenized_stem,main_pos
0,0,조아요 처음구입 싸게햇어요,2,조아요 처음 구입 싸게 햇 어 요,조아요 처음 구입 싸게 햇 요
1,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...,1,생각 보다 잘 안 돼요 매지 바른지 하루 밖에 안 됐는데 ㅠㅠ 25천원 가량 주고 ...,생각 보다 잘 안 돼요 매지 바른지 하루 안 됐는데 ㅠㅠ 25천원 가량 주고 사기 ...
2,2,디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져...,2,디자인 은 괜찮은데 상품 이 금 이 가서 교환 했는데 두번째 받은 상품 도 까져있고...,디자인 은 괜찮은데 상품 이 금 이 가서 교환 했는데 두번째 받은 상품 도 까져있고...
3,3,기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘...,2,기전 에 이 제품 말고 이마트 트 레이더스 에서만 팔던 프리미엄 제품 을 사용 했었...,기전 이 제품 이마트 트 레이더스 팔던 프리미엄 제품 사용 했었습니다 샘플 써 보고...
4,4,튼튼하고 손목을 잘 받쳐주네요~,5,튼튼하고 손목 을 잘 받쳐주네요 ~,튼튼하고 손목 잘 받쳐주네요


In [2]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              25000 non-null  int64 
 1   reviews         25000 non-null  object
 2   target          25000 non-null  int64 
 3   tokenized_stem  25000 non-null  object
 4   main_pos        24996 non-null  object
dtypes: int64(2), object(3)
memory usage: 976.7+ KB


In [3]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              25000 non-null  int64 
 1   reviews         25000 non-null  object
 2   tokenized_stem  25000 non-null  object
 3   main_pos        24998 non-null  object
dtypes: int64(1), object(3)
memory usage: 781.4+ KB


In [4]:
# okt=Okt()

# def token(dataframe):
#     tokenized = []
#     for sentence in tqdm(dataframe['reviews']):
#         tokens = okt.morphs(sentence)
#         tokenize = " ".join(tokens)
#         tokenized.append(tokenize)
#         dataframe["tokenized_stem"] = pd.DataFrame(tokenized)

# def postagging(dataframe):
#     main_pos = []
#     for sentence in tqdm(dataframe['tokenized_stem']):
#         pos = okt.pos(sentence)
#         main_words = [
#             word_pos[0] for word_pos in pos if word_pos[1] in (
#                 'Adjective',
#                 'Noun',
#                 'Adjective',
#                 'Verb',
#                 'Number',
#                 'KoreanParticle'
#             )
#         ]
#         main_words_str = " ".join(main_words)
#         main_pos.append(main_words_str)
#         dataframe["main_pos"] = pd.DataFrame(main_pos)


# token(train)
# token(test)

# postagging(train)
# postagging(test)

# train.to_csv("train_okt.csv", index=False)
# test.to_csv("test_okt.csv", index=False)

# train.head()

## Preprocessing

In [5]:
y = train["target"]

y.value_counts()

5    10000
2     8000
1     4500
4     2500
Name: target, dtype: int64

In [6]:
if vectorizer == "count":
    vec = CountVectorizer(max_features=max_features)
if vectorizer == "hashing":
    vec = HashingVectorizer()
if vectorizer == "tfidf":
    vec = TfidfVectorizer(max_features=max_features)

X = vec.fit_transform(train[feature_name])
X_test = vec.transform(test[feature_name])

X.shape, X_test.shape

((25000, 33414), (25000, 33414))

## Training

In [7]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=validation_size, random_state=seed, stratify=y
)

model = LogisticRegression(solver="liblinear", random_state=seed, n_jobs=-1) 
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
   
val_acc = accuracy_score(y_val, y_pred)
print(f"val_acc : {val_acc}")
wandb.log({"val_acc": val_acc})

val_acc : 0.6488


In [8]:
model = LogisticRegression(solver="liblinear", random_state=seed, n_jobs=-1) 
model.fit(X, y)
prediction = model.predict(X_test)

pd.Series(prediction).value_counts()

5    12518
2    10529
1     1838
4      115
dtype: int64

## Inference

In [9]:
submission = pd.read_csv("data/sample_submission.csv")
submission["target"] = prediction
submission.to_csv("submission.csv", index=False)